# Relationship Analysis Agent 테스트

관계 분석 에이전트 (Level 2) 테스트 노트북

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
서진과 이민호는 한때 가장 친한 친구였다. 어릴 때부터 함께 검술을 배우며 자랐다.
하지만 이민호가 마을을 배신한 후, 그들은 적이 되었다.

하나는 서진의 오랜 동료이자 마을의 치료사였다. 그녀는 항상 서진을 걱정했다.

박서연은 마을의 정탐꾼으로, 서진과 하나를 도와 정보를 수집했다. 
그녀는 하나를 언니처럼 따랐다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_characters": [
                {"name": "서진", "role": "protagonist", "traits": ["brave", "skilled"]},
                {"name": "하나", "role": "supporting", "traits": ["wise", "calm"]},
                {"name": "이민호", "role": "antagonist", "traits": ["bitter", "angry"]},
                {"name": "박서연", "role": "supporting", "traits": ["agile", "loyal"]}
            ],
            "extracted_events": [],
            "relationship_graph": {}, "consistency_report": {}, "retry_count": 0,
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

In [ ]:
from app.agents.analysis.relationship import relationship_analysis_node

async def test_relationship_analysis():
    print("🔗 Relationship Analysis 테스트...")
    state = create_base_state()
    return await relationship_analysis_node(state)

result = run_async(test_relationship_analysis())
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
# Re-analysis 테스트
async def test_re_analysis():
    print("🔄 Re-analysis 테스트...")
    state = create_base_state()
    state["retry_count"] = 1
    state["relationship_graph"] = {"relationships": [{"source": "서진", "target": "이민호", "type": "FRIENDLY"}]}
    state["consistency_report"] = {"conflicts": [{"type": "RELATIONSHIP_CONFLICT", "description": "배신 후 FRIENDLY는 부적절"}]}
    return await relationship_analysis_node(state)

re_result = run_async(test_re_analysis())
print(json.dumps(re_result, ensure_ascii=False, indent=2))